# Building a Spam Filter with Naive Bayes

In this project, we will be building a spam filter for SMS messages. Our first step will be to "teach" the computer how to classify messages. This will be accomplished by writing an algorithm to look at the probability of whether a message is spam or ham (not spam) by looking at the probability that each word in a message is found in a spam message or a ham message. We'll use a [Naive Bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) algorithm to calculate this probability and develop our algorithm using a [dateset of 5,272 SMS messages](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection) that have already been classified by humans.

Once we have developed our algorithm, we will test it using the same dataset.

## Exploring the Dataset

To begin, we'll import the dataset as a Pandas dataframe object and explore it a bit to understand it before proceeding.

In [1]:
import pandas as pd

messages = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
Label    5572 non-null object
SMS      5572 non-null object
dtypes: object(2)
memory usage: 87.1+ KB


In [2]:
messages.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
messages['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

## Training and Test Set

We can see that in this particular dataset, approximately 87% of the messages are not spam (ham). Since we will be using the same dataset to both train and test our spam filter, we need to split the dataset into two different pieces: one for training and one for testing. We will use 80% of the dataset (4,458 messages) to develop our filter and the remaining 20% (1,114) to test it.

In order to ensure that our two sets are sufficiently random, we will randomize the messages in the original dataset before splitting it into two pieces.

In [4]:
messages_random = messages.sample(frac=1, random_state=1)
messages_random.head()

,Label,SMS
1078,ham,"Yep, by the pretty sculpture"
4028,ham,"Yes, princess. Are you going to make me moan?"
958,ham,Welp apparently he retired
4642,ham,Havent.
4674,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [5]:
training = messages_random.iloc[:4458]
training.reset_index(drop=True, inplace=True)
training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4458 entries, 0 to 4457
Data columns (total 2 columns):
Label    4458 non-null object
SMS      4458 non-null object
dtypes: object(2)
memory usage: 69.7+ KB


In [6]:
test = messages_random.iloc[4458:]
test.reset_index(drop=True, inplace=True)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1114 entries, 0 to 1113
Data columns (total 2 columns):
Label    1114 non-null object
SMS      1114 non-null object
dtypes: object(2)
memory usage: 17.5+ KB


In [7]:
print(training['Label'].value_counts(normalize=True))
print('\n')
print(test['Label'].value_counts(normalize=True))

ham     0.86541
spam    0.13459
Name: Label, dtype: float64


ham     0.868043
spam    0.131957
Name: Label, dtype: float64


We can see that these two sets are suitable for our purposes as the percentage of ham and spam messages are nearly identical.

## Letter Case and Punctuation

With our dataset successfully split into a training set and a testing set, we can begin the process of developing an algorithm to successfully classify new messages. In order to classify messages, we will be using the following Naive Bayes equations:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

Essentially, we will be looking at the likelihood that a message is either spam or ham given that it contains certain words by looking at the overall probability that a message is either spam or ham and multiplying that by a factor that considers how likely a particular word is to be in either a spam or a ham message. To save time and computing power, we will not be calculating the actual probability that a message is either spam or ham, but we can use the relative values of the two equations above to find into which category a message is more likely to fall. In order to determine the probability that a word is in either a spam or ham message, we will use the following equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

These equations look at the number of times a particular word occurs in a spam and/or ham message and divides this by the total number of words in each category of message. Because a word may only occur in one category, we will use [Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) so that we can never end up with 0 (zero).

In order to achieve these calculations, we need to split each SMS message into single words. To begin this process, we will remove all punctuation and capitalization from the messages. This will create uniformity of vocabulary and prevent `Yes` and `yes` from being categorized as two different words.

In [8]:
import re

def strip_char(msg):
    return re.sub(r'[^\w]',' ',msg).lower()

training_parsed = training.copy()
training_parsed['SMS'] = training['SMS'].apply(strip_char)
training_parsed.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


## Creating the Vocabulary

Now that we have the messages cleaned, we need to split them into lists of words and create a vocabulary of unique words to use in our calculations.

In [9]:
def create_list(msg):
    return msg.split()

training_parsed['SMS'] = training_parsed['SMS'].apply(create_list)
training_parsed.head()  

,Label,SMS
0,ham,"[yep, by, the, pretty, sculpture]"
1,ham,"[yes, princess, are, you, going, to, make, me,..."
2,ham,"[welp, apparently, he, retired]"
3,ham,[havent]
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,..."


In [10]:
vocabulary = []

for row in training_parsed['SMS']:
    for item in row:
        vocabulary.append(item)

# Remove the duplicates from the list
vocabulary = set(vocabulary)
vocabulary = list(vocabulary)

print('Total number of unique words: ' + str(len(vocabulary)))

Total number of unique words: 7783


## The Final Training Set

Our final task in preparing the training set is to create columns for each of the unique words and assign the value of each column to the number of times that unique word is in that particular SMS.

In [11]:
# Create a dictionary with a key for every unique word and a list of values for each row in the dataset
word_counts_per_sms = {unique_word: [0] * len(training_parsed['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_parsed['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [12]:
# Convert the dictionary to a dataframe object
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [13]:
# Combine the two dataframes
training_set = pd.concat([training_parsed, word_counts], axis=1)
training_set.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## Calculating Constants First

With the dataset prepared, we can begin the creation of the spam filter. The first step is to create the constants we will use in our calculations. We need to know the probability that a message is spam or ham as well as the number of words in each category of message.

In [14]:
training_set['WordCount'] = training_set['SMS'].apply(
    lambda sms: len(sms))
training_set.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥,WordCount
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,26


In [15]:
training_spam = training_set[training_set['Label'] == 'spam']
training_ham = training_set[training_set['Label'] == 'ham']

# Probability a message is either spam or ham
p_spam = len(training_spam) / len(training_set)
p_ham = len(training_ham) / len(training_set)

# Number of words in each category of message
n_spam = sum(training_spam['WordCount'])
n_ham = sum(training_ham['WordCount'])
n_words = n_spam + n_ham

## Calculating Parameters

Our next step is to calculate the probability that each unique word is in a spam message or in a ham message. Recall that we are using the following equations to do so.

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

Rather than calculating the probability each time we classify the message, we will calculate the probabiliity once and save the data. This saves a huge number of calculations and makes processing much quicker. We will be using a smoothing factor of 1 (one).

In [16]:
# Create a dictionary of each unique word for each type of message
spam_dict = {}
ham_dict = {}

for row in training_spam['SMS']:
    for item in row:
        if item not in spam_dict:
            spam_dict[item] = 0

for row in training_ham['SMS']:
    for item in row:
        if item not in ham_dict:
            ham_dict[item] = 0


In [17]:
# Smoothing factor
alpha = 1
unique_words = len(vocabulary)

# Calculate the probability that each word is in a spam or ham message and assign it to the dictionaries
for word in vocabulary:
    spam_numerator = sum(training_spam[word]) + alpha
    ham_numerator = sum(training_ham[word]) + alpha

    spam_denominator = n_spam + unique_words
    ham_denominator = n_ham + unique_words
    
    spam_dict[word] = spam_numerator / spam_denominator
    ham_dict[word] = ham_numerator / ham_denominator

## Classifying a New Message

With all of the constants and parameters calculated, all that is left is to create the actual filter. We are using the following equations:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

Recall that we are not calculating the actual probability that a message is either ham or spam. The message is classified by whichever value is larger.

In [18]:
def classify(message):

    message = re.sub(r'[^\w]', ' ', message)
    message = message.lower()
    message = message.split()
   
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in spam_dict:
            p_spam_given_message *= spam_dict[word]
        if word in ham_dict:
            p_ham_given_message *= ham_dict[word]
        
    
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

We will test two sample messages, one that is obviously spam, and one that is obviously ham.

In [19]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [20]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Measuring the Spam Filter's Accuracy

The filter worked correctly for our sample messages. Our next step is to see how well the filter performs on our test set of 1,114 messages. We will create a new column, `Classification`, that contains the output of our filtering method. We can then use this column to see how well our filter classified messages compared to the human filtering already performed on our dataset.

In [21]:
def classify_message(message):

    message = re.sub(r'[^\w]', ' ', message)
    message = message.lower()
    message = message.split()
   
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in spam_dict:
            p_spam_given_message *= spam_dict[word]
        if word in ham_dict:
            p_ham_given_message *= ham_dict[word]
        
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    # If the two values are equal, the message needs to be analyzed by a human
    else:
        return 'needs human classification'


In [22]:
test_set = test.copy()
test_set['Classification'] = test['SMS'].apply(classify_message)
test_set['Classification'].value_counts(normalize=True)

ham                           0.869838
spam                          0.129264
needs human classification    0.000898
Name: Classification, dtype: float64

Our initial values are similar to the original proportions of ham and spam. Let's calculate what percentage of messages our algorithm classifies the same as the human classification.

In [23]:
correct = 0
total = len(test_set)

for row in test_set.itertuples():
    if(row[1] == row[3]):
        correct += 1

print('Correct classification: ' + str(round( 100 * correct / total, 2)) + '%')

Correct classification: 98.74%


## Conclusions

Our filter is accurate nearly 99% of the time! There are a handful of messages which could be analyzed in order to potentially improve the accuracy, but this is an excellent result.